#### Test

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [47]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV


In [48]:
random_seed = 42
np.random.seed(random_seed)

In [49]:
# prepare white wine data, with pandas
wine_data = pd.read_csv("winequality-white.csv", delimiter=";")
# print(wine_data, type(wine_data))


In [50]:
wine_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [51]:
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


#### Imputing
Do we need imputing?

#### test train split

In [52]:
wine_data['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [53]:
# transform wine quality into 3 categories
bins = [0, 4, 6, 10]  # Specify the bin edges
labels = [1, 2, 3]    # Specify the labels for each category
wine_data["new_quality"] = pd.cut(wine_data["quality"], bins=bins, labels=labels, include_lowest=True)


In [54]:
wine_output = wine_data["new_quality"]
wine_feature = wine_data.drop(columns=["quality", "new_quality"])

In [55]:
wine_data['new_quality'].value_counts()

2    3655
3    1060
1     183
Name: new_quality, dtype: int64

In [56]:
wine_output.describe()

count     4898
unique       3
top          2
freq      3655
Name: new_quality, dtype: int64

In [57]:
wine_feature.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000


In [58]:
# train and test split
test_size_ratio = 0.2
wine_feature_train, wine_feature_test, wine_output_train, wine_output_test = train_test_split(wine_feature, wine_output, test_size=test_size_ratio, random_state=random_seed)


In [59]:
wine_feature_train.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,6.865046,0.279338,0.332731,6.450702,0.045734,35.094564,138.001149,0.994071,3.189293,0.489781,10.508840
std,0.844483,0.101606,0.119758,5.139311,0.021797,16.676958,42.067667,0.003022,0.150183,0.113590,1.227887
min,3.800000,0.080000,0.000000,0.600000,0.009000,3.000000,10.000000,0.987110,2.720000,0.220000,8.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991740,3.090000,0.410000,9.500000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,33.000000,134.000000,0.993800,3.180000,0.470000,10.400000
75%,7.300000,0.330000,0.380000,10.000000,0.050000,46.000000,167.000000,0.996200,3.280000,0.550000,11.400000
max,11.800000,1.100000,1.660000,65.800000,0.346000,146.500000,313.000000,1.038980,3.820000,1.080000,14.200000


In [60]:
wine_feature_test.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,6.813776,0.273857,0.340031,6.154388,0.045926,36.161735,139.797959,0.993854,3.184163,0.490112,10.535963
std,0.840584,0.097411,0.125833,4.788953,0.022059,18.251705,44.169785,0.002859,0.154235,0.116302,1.241884
min,3.900000,0.080000,0.000000,0.700000,0.014000,2.000000,9.000000,0.987220,2.800000,0.250000,8.000000
25%,6.200000,0.210000,0.270000,1.700000,0.036000,24.000000,110.000000,0.991650,3.080000,0.410000,9.500000
50%,6.800000,0.260000,0.320000,5.000000,0.042000,34.500000,134.000000,0.993655,3.170000,0.480000,10.400000
75%,7.300000,0.320000,0.390000,9.325000,0.050000,46.250000,168.000000,0.995970,3.270000,0.550000,11.400000
max,14.200000,0.760000,0.990000,22.600000,0.240000,289.000000,440.000000,1.001000,3.800000,1.010000,14.000000


In [61]:
wine_output_train.describe()


count     3918
unique       3
top          2
freq      2932
Name: new_quality, dtype: int64

In [62]:
wine_output_train.value_counts()

2    2932
3     833
1     153
Name: new_quality, dtype: int64

In [63]:
wine_output_test.describe()

count     980
unique      3
top         2
freq      723
Name: new_quality, dtype: int64

In [64]:
wine_output_test.value_counts()

2    723
3    227
1     30
Name: new_quality, dtype: int64

### Logistic Regression

In [65]:
# logistic regression model
log_reg = LogisticRegression(max_iter=50_000)

log_reg.fit(wine_feature_train, wine_output_train)

LogisticRegression(max_iter=50000)

In [68]:
log_reg_train_score = log_reg.score(wine_feature_train, wine_output_train)
log_reg_test_score = log_reg.score(wine_feature_test, wine_output_test)

print(f"logistic regression training score:  {log_reg_train_score} ")
print(f"logistic regression test score:  {log_reg_test_score} ")

logistic regression training score:  0.7751403777437468 
logistic regression test score:  0.7571428571428571 
